In [1]:
from platform import python_version
# !pip install shap==0.39
# !pip install transformers==4.5
python_version()

'3.7.13'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from utils import HuggingfaceDatasetManager, DatasetTypeEnum, explain_text, LABEL_MAPPINGS
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
model_name = "chinhon/fake_tweet_detect"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, binary_output=True, device=0)
embedding_dim = model.distilbert.embeddings.word_embeddings.embedding_dim

In [6]:
dataset_type = DatasetTypeEnum.CHINHON_FAKE_TWEET_DETECT

In [7]:
dataset_manager = HuggingfaceDatasetManager(dataset_type=dataset_type, embedding_dim=embedding_dim)

Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/real_50k.csv
Loaded 50000 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/real_50k.csv
Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/troll_50k.csv
Loaded 50000 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/troll_50k.csv


In [8]:
true_samples = dataset_manager.fetch_true_samples(sample_count=10)
true_sample_shap_values = explain_text(dataset_type=dataset_type, pipeline=pipeline, text=true_samples[0])

Getting the following indexes: [39064 11110 45386 33239 38193 36289 28091  9349 21428 30403]
Explaining the following text: 
<--------------------------------------------------------------------------------------------->
I am undeserving of such honor and am grateful for your criticism I can do better #StruggleSession
<--------------------------------------------------------------------------------------------->
Predicted True with score: 0.8334335088729858
Predicted Fake with score: 0.16656655073165894


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
pipeline([true_samples[0]])

[[{'label': 'LABEL_0', 'score': 0.8334335088729858},
  {'label': 'LABEL_1', 'score': 0.16656655073165894}]]